# Introduction / Business Problem

My partner is a jewellery designer/maker, with a successful shop in one of the trendier parts of London.  She has in the past expressed an interest in opening a second shop, and her usual methodology for this might be to go do some market research.  Rather than simply ask the coolest people we know, I’d like to use data science to identify an appropriate location.   

The basic question then is this: can we work out which are the hippest areas in London, and the areas that are on the verge of becoming hip?  And given that where would be a good place to open a cool jewelry shop?

This approach could more broadly apply to anyone interested in opening an independent store or brand in London, and if the approach is successful might be more broadly applied to any city for which we have the appropriate data.


# Data

We'll break this down into 2 components: quantifying trendiness into clusters, and finding competitive advantage.

To quantify trendiness we'll explore different options, among them identifying whether there are a large number of chain (non-independent) stores in the area, or stores with particular or unusual categories.  We'll look at the frequency and density of specific category types such as record stores and coffee shops.

To finding competitive advantage, we'll look at details of the specific jewelry shops, including rating and price.  Consideration should be given to the number of total shops and restaurants in the area (which would generate more footfall and be a positive influence), as well as other jewellery shops (which would mean more competition and be a negative influence).

The data we'll need includes:

* **Geocoding** - to identify the specific locations
* **Foursquare API** - to identify the qualifying places (jewelry stores, restaurants etc.) and essential details around them


### Geocoding

[Geonames.org](https://www.geonames.org/) is a fantastic free resource that publishes geocoded names and their latitudes/longitudes many countries around the world.  The GeoNames geographical database is available for download free of charge under a creative commons attribution license.  The resource includes an explict accuracy for the latitude/longitude, which will be useful for filtering out bad/inaccurate/duplicate data.


### Foursquare API

We will retrieve both the places and necessary details from the Foursquare API.  We'll get the precise location and category of the shop, as well as details such as price and rating where necessary.  Some of these calls will be necessarily premium calls, so to keep ourselves within the boundaries of free usage we'll limit them.


